In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
data1 = pd.read_csv('part_2_XYZ_RFM_SALES_final_data_nov_27_invent_class.csv')
data2 = pd.read_csv('part_2_XYZ_RFM_SALES_final_data_nov_27_invent_class.csv')

In [3]:
data2['invoicedate'] = pd.to_datetime(data2['invoicedate'])
data1['invoicedate'] = pd.to_datetime(data1['invoicedate'])

In [4]:
data2.head()

,invoiceno,code,quantity,year,invoicedate,unitprice,sellprice,custid,country,shipper,...,region,salesperson,margin,commodity,absmargin,month,count,class,class_type,invent_class
0,568200,PADS,5,2017,2017-09-25,0.999975,4.999876,16198,United Kingdom,Delivery Truck,...,SOUTH,GRACE J,0,Studs,0.000113,9,1,C,CZ,red
1,568375,BANK CHARGES,5,2017,2017-09-26,0.999977,4.999883,13405,United Kingdom,UPS Next day Air,...,WEST,JEFF Y,100,Studs,4.999883,9,1,C,CZ,red
2,561226,PADS,5,2017,2017-07-26,0.999886,4.999428,15618,United Kingdom,Delivery Truck,...,SOUTH,GRACE J,0,Studs,-0.000335,7,1,C,CZ,red
3,2550193,PADS,7,2018,2018-06-08,0.999962,6.999731,13952,United Kingdom,UPS ground 2nd day,...,CENTRAL,PETER T,0,Studs,0.000063,6,1,C,CZ,red
4,550193,PADS,5,2017,2017-04-15,0.999988,4.999940,13952,United Kingdom,UPS ground 2nd day,...,CENTRAL,PETER T,0,Studs,0.000177,4,1,C,CZ,red


In [5]:
data1.head()

,invoiceno,code,quantity,year,invoicedate,unitprice,sellprice,custid,country,shipper,...,region,salesperson,margin,commodity,absmargin,month,count,class,class_type,invent_class
0,568200,PADS,5,2017,2017-09-25,0.999975,4.999876,16198,United Kingdom,Delivery Truck,...,SOUTH,GRACE J,0,Studs,0.000113,9,1,C,CZ,red
1,568375,BANK CHARGES,5,2017,2017-09-26,0.999977,4.999883,13405,United Kingdom,UPS Next day Air,...,WEST,JEFF Y,100,Studs,4.999883,9,1,C,CZ,red
2,561226,PADS,5,2017,2017-07-26,0.999886,4.999428,15618,United Kingdom,Delivery Truck,...,SOUTH,GRACE J,0,Studs,-0.000335,7,1,C,CZ,red
3,2550193,PADS,7,2018,2018-06-08,0.999962,6.999731,13952,United Kingdom,UPS ground 2nd day,...,CENTRAL,PETER T,0,Studs,0.000063,6,1,C,CZ,red
4,550193,PADS,5,2017,2017-04-15,0.999988,4.999940,13952,United Kingdom,UPS ground 2nd day,...,CENTRAL,PETER T,0,Studs,0.000177,4,1,C,CZ,red


In [6]:
data1['invoicedate'][1]

Timestamp('2017-09-26 00:00:00')

In [7]:

now = datetime.datetime.now()
print(now)

2019-12-02 00:02:36.637596


In [8]:
rfm = data1.groupby('custid', as_index = False).agg({'invoicedate': lambda x: (now - x.max()).days, 'invoiceno': lambda x: len(x.unique()), 'absmargin': lambda x: x.sum(), 'sellprice': lambda x:x.sum()})
rfm['invoicedate'] = rfm['invoicedate'].astype(int)
rfm.rename(columns={'invoicedate': 'recency', 
                         'invoiceno': 'frequency', 
                         'sellprice': 'monetary',
                         'absmargin': 'monetary_margin'}, inplace=True)

In [9]:
rfm.head()

,custid,recency,frequency,monetary_margin,monetary
0,12346,572,2,-104578.534125,750866.973072
1,12347,361,10,1064.264409,22627.316532
2,12348,352,7,93.741039,20091.821036
3,12349,741,1,131.620274,959.745973
4,12350,488,2,-10.605872,987.687845


In [10]:
rfm['recency_log'] = np.log(rfm['recency'])
rfm['frequency_log'] = np.log(rfm['frequency'])
rfm['monetary_log'] = np.log(rfm['monetary'])

rfm['monetary_margin_log'] = np.log(rfm['monetary_margin'] + 104578.53418627633 + 1) 

In [11]:
drfm1 = rfm[['custid','recency','frequency_log','monetary_log']]

In [12]:
quantiles = drfm1.quantile(q=[0.25,0.50,0.75])
quantiles

,custid,recency,frequency_log,monetary_log
0.25,13809.25,363.0,0.693147,6.378600
0.50,15278.00,450.0,1.098612,7.610157
0.75,16772.50,751.0,1.791759,8.804403


In [13]:
quantiles = quantiles.to_dict()
quantiles

{'custid': {0.25: 13809.25, 0.5: 15278.0, 0.75: 16772.5},
 'frequency_log': {0.25: 0.69314718055994529,
  0.5: 1.0986122886681098,
  0.75: 1.791759469228055},
 'monetary_log': {0.25: 6.3785997100264025,
  0.5: 7.6101565452341742,
  0.75: 8.80440250911861},
 'recency': {0.25: 363.0, 0.5: 450.0, 0.75: 751.0}}

In [14]:
drfm1.head()

,custid,recency,frequency_log,monetary_log
0,12346,572,0.693147,13.528984
1,12347,361,2.302585,10.026913
2,12348,352,1.945910,9.908068
3,12349,741,0.000000,6.866669
4,12350,488,0.693147,6.895367


In [15]:
segmented_rfm = drfm1

In [16]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [17]:
segmented_rfm['R'] = segmented_rfm['recency'].apply(lambda x: RScore(x,'recency',quantiles))
segmented_rfm['F'] = segmented_rfm['frequency_log'].apply(lambda x: FScore(x,'frequency_log',quantiles))

segmented_rfm['M'] = segmented_rfm['monetary_log'].apply(lambda x: FScore(x,'monetary_log',quantiles))

D:\ArcGIS\a\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\ArcGIS\a\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\ArcGIS\a\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [18]:
segmented_rfm['score'] = segmented_rfm['R'].map(str) + segmented_rfm['F'].map(str) + segmented_rfm['M'].map(str)
segmented_rfm.head()

D:\ArcGIS\a\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,custid,recency,frequency_log,monetary_log,R,F,M,score
0,12346,572,0.693147,13.528984,3,4,1,341
1,12347,361,2.302585,10.026913,1,1,1,111
2,12348,352,1.945910,9.908068,1,1,1,111
3,12349,741,0.000000,6.866669,3,4,3,343
4,12350,488,0.693147,6.895367,3,4,3,343


In [19]:
segmented_rfm.head()

,custid,recency,frequency_log,monetary_log,R,F,M,score
0,12346,572,0.693147,13.528984,3,4,1,341
1,12347,361,2.302585,10.026913,1,1,1,111
2,12348,352,1.945910,9.908068,1,1,1,111
3,12349,741,0.000000,6.866669,3,4,3,343
4,12350,488,0.693147,6.895367,3,4,3,343


### Customer Category segmentation based on the RFM VALUES

In [20]:
segmented_rfm.loc[(segmented_rfm['R'].isin(['1','2','3','4']))& (segmented_rfm['F'].isin(['1','2'])) & (segmented_rfm['M'].isin(['1','2','3','4'])) ,'category'] = 'Highvalue'
segmented_rfm.loc[(segmented_rfm['R'].isin(['1']))& (segmented_rfm['F'].isin(['1'])) & (segmented_rfm['M'].isin(['1'])) ,'category'] = 'Core'
segmented_rfm.loc[(segmented_rfm['R'].isin(['1','2','3','4']))& (segmented_rfm['F'].isin(['3','4'])) & (segmented_rfm['M'].isin(['1','2','3','4'])) ,'category'] = 'Hibernating'
segmented_rfm.loc[(segmented_rfm['R'].isin(['3','4']))& (segmented_rfm['F'].isin(['1','2','3','4'])) & (segmented_rfm['M'].isin(['1','2'])) ,'category'] = 'At Risk'


In [21]:
try2 = segmented_rfm[['custid','R','F','M','score','category']]

In [22]:
all_added = pd.merge(data2,try2,on='custid',how='left')

In [23]:
all_added.head()

,invoiceno,code,quantity,year,invoicedate,unitprice,sellprice,custid,country,shipper,...,month,count,class,class_type,invent_class,R,F,M,score,category
0,568200,PADS,5,2017,2017-09-25,0.999975,4.999876,16198,United Kingdom,Delivery Truck,...,9,1,C,CZ,red,4,4,4,444,Hibernating
1,568375,BANK CHARGES,5,2017,2017-09-26,0.999977,4.999883,13405,United Kingdom,UPS Next day Air,...,9,1,C,CZ,red,2,2,2,222,Highvalue
2,561226,PADS,5,2017,2017-07-26,0.999886,4.999428,15618,United Kingdom,Delivery Truck,...,7,1,C,CZ,red,4,2,4,424,Highvalue
3,2550193,PADS,7,2018,2018-06-08,0.999962,6.999731,13952,United Kingdom,UPS ground 2nd day,...,6,1,C,CZ,red,2,2,1,221,Highvalue
4,550193,PADS,5,2017,2017-04-15,0.999988,4.999940,13952,United Kingdom,UPS ground 2nd day,...,4,1,C,CZ,red,2,2,1,221,Highvalue


In [24]:
all_added.to_csv('part_3_RFM_SCORE_sales_data_with_customer_RFM_score_nov_27_final.csv',index=False)